In [1]:
import os
import numpy as np
from matplotlib.lines import Line2D
from scipy.stats import pearsonr, gaussian_kde, linregress, ttest_ind, sem, zscore
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import norm
from scipy.stats import percentileofscore
from sklearn.utils.validation import check_random_state
from math import factorial
from more_itertools import distinct_permutations
import statsmodels.api as sm

import matplotlib.pyplot as plt

import pandas as pd
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats

import numpy as np
from scipy.stats import linregress
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import KFold, ParameterGrid, train_test_split
from tqdm.notebook import tqdm

import warnings
import random
#from torch.utils.data import SubsetRandomSampler
from sklearn.utils import resample

from sklearn.preprocessing import MinMaxScaler
from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score
from collections import Counter

from statsmodels.stats.diagnostic import linear_harvey_collier
from scipy.stats import chi2
from sklearn.model_selection import train_test_split
from colorama import Fore, Style, init
from sklearn.metrics import roc_auc_score

warnings.filterwarnings("ignore")



In [2]:
from scipy.stats import chi2

def summarize_feature(df, feature, df_out, stat_cols=(10, 50), df_offset=2):

    d = df[df["Feature"] == feature]
    if d.empty:
        return df_out  

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            combined_p = 1 - chi2.cdf(d[col].mean(), df_chi)
            df_out.loc[feature, f"hl_pvalue_combined_{k}"] = combined_p

    # --- OR summary ---
    if "OR" in d.columns:
        or_mean = d["OR"].mean()
        or_std = d["OR"].std(ddof=1)
        df_out.loc[feature, "OR"] = or_mean
        df_out.loc[feature, "2.5%"] = or_mean - or_std
        df_out.loc[feature, "97.5%"] = or_mean + or_std

    # --- min/max de los IC originales (tal como lo estabas haciendo) ---
    if "2.5%" in d.columns:
        df_out.loc[feature, "minOR"] = d["2.5%"].min()
    if "97.5%" in d.columns:
        df_out.loc[feature, "maxOR"] = d["97.5%"].max()

    return df_out


features = ["Mono", "One", "Two", "Three", "Total"] 




def hosmer_lemeshow(y_true, y_prob, g):
    hl_df = pd.DataFrame({
        "observed": y_true,
        "predicted_probability": y_prob
    }).dropna()


    hl_df["group"] = pd.qcut(hl_df["predicted_probability"], g, duplicates="drop")

    hl_table = hl_df.groupby("group").apply(
        lambda x: pd.Series({
            "observed": x["observed"].sum(),
            "expected": x["predicted_probability"].sum(),
            "total": len(x)
        })
    )

    hl_table["observed_neg"] = hl_table["total"] - hl_table["observed"]
    hl_table["expected_neg"] = hl_table["total"] - hl_table["expected"]


    hl_statistic = (
        ((hl_table["observed"] - hl_table["expected"])**2) / hl_table["expected"] +
        ((hl_table["observed_neg"] - hl_table["expected_neg"])**2) / hl_table["expected_neg"]
    ).sum()


    dof = hl_table.shape[0] - 2
    p_value = 1 - chi2.cdf(hl_statistic, dof)

    return hl_statistic, p_value



def summarize_feature_by_covar(df, covar, feature, df_out, stat_cols=(10, 50), df_offset=2):
    d = df[(df["Covar"] == covar) & (df["Feature"] == feature)]
    if d.empty:
        return df_out

    idx = (covar, feature)

    for k in stat_cols:
        col = f"hl_statistic_{k}"
        if col in d.columns:
            df_chi = k - df_offset
            df_out.loc[idx, f"hl_pvalue_combined_{k}"] = 1 - chi2.cdf(d[col].mean(), df_chi)

    or_mean = d["OR"].mean()
    or_std  = d["OR"].std(ddof=1)
    df_out.loc[idx, "OR"] = or_mean
    df_out.loc[idx, "2.5%"] = or_mean - or_std
    df_out.loc[idx, "97.5%"] = or_mean + or_std

    df_out.loc[idx, "minOR"] = d["2.5%"].min()
    df_out.loc[idx, "maxOR"] = d["97.5%"].max()

    df_out.loc[idx, "n_iter"] = d.shape[0]

    return df_out


## Load data

In [3]:
import time

t0 = time.perf_counter()

In [4]:
data = pd.read_csv('data/BBAG-cross.csv')

In [5]:
vars_ = ['Mono', 'One',	'Two',	'Three', 'Total']


# Odds ratios

## Without co-vars

### Without iterations

In [6]:
results_merge_df_all = data.copy()

results_merge_df_all = results_merge_df_all.loc[:, ~results_merge_df_all.columns.duplicated()]

df_directions_odd = pd.DataFrame()

for i in vars_:
    
    y_ols = results_merge_df_all['GAP_bin']

    
    X_ols = results_merge_df_all[[i]]
    X_train, X_test, y_train, y_test = train_test_split(X_ols, y_ols, test_size=0.2, random_state=42)
    

    scaler = MinMaxScaler((0.05, 0.95))
    X_train_scaled = scaler.fit_transform(X_train)
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
    
    X_test_scaled = scaler.transform(X_test)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)
    
    X_train_scaled['intercept'] = 1
    X_test_scaled['intercept'] = 1
    
    model = sm.Logit(y_train, X_train_scaled).fit(disp = 0)
    
    params = model.params
    conf = np.exp(model.conf_int())
    conf['OR'] = np.exp(params)
    conf['z'] =model.tvalues
    conf['P>|z|'] =model.pvalues
    conf.columns = ['2.5%', '97.5%', 'OR', 'z', 'P>|z|']
    
    df = conf.loc[i:i]
    df['Feature'] = i
    df_directions_odd = pd.concat([df_directions_odd, df])

df_directions_odd = df_directions_odd.reset_index(drop=True)

df_directions_odd.to_excel('Results/cross/cross_OR_-with-slovakia.xlsx')
df_directions_odd

,2.5%,97.5%,OR,z,P>|z|,Feature
0,2.135939,2.417376,2.272305,25.994122,5.771115e-149,Mono
1,0.762767,0.890133,0.823993,-4.914371,8.906794e-07,One
2,0.441877,0.512197,0.475740,-19.718974,1.482049e-86,Two
3,0.570497,0.661718,0.614417,-12.871924,6.476807e-38,Three
4,0.413571,0.468108,0.439996,-25.980834,8.155405e-149,Total


### 1000-iteration

In [7]:


count = 0

results_dict = {}
for i in vars_:
    results_dict[i + "_y"] = pd.Series(dtype=float)
    results_dict[i + "_ypred"] = pd.Series(dtype=float)

results_merge_df_all = data.copy()
results_merge_df_all.dropna(subset=vars_ + ["GAP_bin"], inplace=True)
results_merge_df_all.reset_index(inplace=True, drop=True)

df_directions_odd = pd.DataFrame()

n_boosts = 10  #For performance reasons, the number of iterations was set to 10; however, 1,000 iterations were used for the results reported in the manuscript.”
test_size = 500 / results_merge_df_all.shape[0]

for boosts in range(n_boosts):
    for i in vars_:

        y_ols = results_merge_df_all["GAP_bin"]
        X_ols = results_merge_df_all[[i]]

        X_train, X_test, y_train, y_test = train_test_split(
            X_ols, y_ols,
            test_size=test_size,
            stratify=results_merge_df_all["GAP_bin"],
            random_state=boosts
        )

        scaler = MinMaxScaler((0.05, 0.95))

        X_train_scaled = pd.DataFrame(
            scaler.fit_transform(X_train),
            columns=X_train.columns,
            index=X_train.index
        )
        X_test_scaled = pd.DataFrame(
            scaler.transform(X_test),
            columns=X_test.columns,
            index=X_test.index
        )

        X_train_scaled["intercept"] = 1
        X_test_scaled["intercept"] = 1


        model = sm.Logit(y_train, X_train_scaled).fit(disp=0)


        y_test_pred = model.predict(X_test_scaled)

        results_dict[i + "_y"] = pd.concat([results_dict[i + "_y"], y_test], axis=0)
        results_dict[i + "_ypred"] = pd.concat([results_dict[i + "_ypred"], y_test_pred], axis=0)

        params = model.params
        conf = np.exp(model.conf_int())
        conf["OR"] = np.exp(params)
        conf["z"] = model.tvalues
        conf["P>|z|"] = model.pvalues
        conf.columns = ["2.5%", "97.5%", "OR", "z", "P>|z|"]

        df = conf.loc[i:i].copy()
        df["Feature"] = i

        hl10_stat, hl10_p = hosmer_lemeshow(y_test, y_test_pred, g=10)
        hl50_stat, hl50_p = hosmer_lemeshow(y_test, y_test_pred, g=50)
        hl100_stat, hl100_p = hosmer_lemeshow(y_test, y_test_pred, g=100)

        df["hl_statistic_10"] = hl10_stat
        df["hl_pvalue_10"] = hl10_p

        df["hl_statistic_50"] = hl50_stat
        df["hl_pvalue_50"] = hl50_p

        df["hl_statistic_100"] = hl100_stat
        df["hl_pvalue_100"] = hl100_p

        df_directions_odd = pd.concat([df_directions_odd, df], axis=0)

    df_directions_odd = df_directions_odd.reset_index(drop=True)

df_directions_odd;


In [8]:
df_directions_odd_f = pd.DataFrame()

for f in vars_:
    df_directions_odd_f = summarize_feature(df_directions_odd, f, df_directions_odd_f, stat_cols=(10, 50, 100))

df_directions_odd_f.to_excel('Results/cross/cross_OR_-with-slovakia-iter.xlsx')
df_directions_odd_f

,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR
Mono,0.232323,0.186187,0.970193,2.245776,2.240748,2.250804,2.119085,2.382944
One,0.120756,0.310726,0.985922,0.806992,0.805486,0.808498,0.750194,0.867732
Two,0.252713,0.254522,0.982004,0.491265,0.490420,0.492109,0.458981,0.526322
Three,0.084505,0.288554,0.990875,0.620513,0.618864,0.622161,0.577527,0.665406
Total,0.226202,0.202770,0.975630,0.445175,0.444178,0.446171,0.419528,0.471808


## Odd ratios with macrosocials

### Without Iteration

In [9]:
vars_social_physical = ['gender_equal_l', 'Polution_conc_inv', 'Eq']

vars_sociopolitical = ['free_parties_l',  'inclu_suff_est', 'cred_elect_est', 'local_dem_est']

In [10]:
results_merge_df_all = data.copy()

covar_list = (
    ['total_exposomes', 'sociopolitical', 'social_physical', 'GDP',
     'number_leng_inst', 'number_stable_inst', 'distance', 'Migration']
    + vars_social_physical
    + vars_sociopolitical
)

df_directions_odd_all = []
df_directions_cov_odd_all = []

for covar in covar_list:
    for i in vars_:

        y_ols = results_merge_df_all['GAP_bin']
        X_ols = results_merge_df_all[[i, covar]]

        X_train, X_test, y_train, y_test = train_test_split(
            X_ols, y_ols, test_size=0.2, random_state=42
        )

        scaler = MinMaxScaler((0.05, 0.95))

        X_train_scaled = pd.DataFrame(
            scaler.fit_transform(X_train),
            columns=X_train.columns,
            index=X_train.index
        )
        X_test_scaled = pd.DataFrame(
            scaler.transform(X_test),
            columns=X_test.columns,
            index=X_test.index
        )

        X_train_scaled['intercept'] = 1
        X_test_scaled['intercept'] = 1

        model = sm.Logit(y_train, X_train_scaled).fit(disp=0)

        params = model.params
        conf = np.exp(model.conf_int())
        conf['OR'] = np.exp(params)
        conf['z'] = model.tvalues
        conf['P>|z|'] = model.pvalues
        conf.columns = ['2.5%', '97.5%', 'OR', 'z', 'P>|z|']

        df_i = conf.loc[i:i].copy()
        df_i['Feature'] = i
        df_i['Covar'] = covar
        df_directions_odd_all.append(df_i)

        df_c = conf.loc[covar:covar].copy()
        df_c['Feature'] = f"{i}({covar})"
        df_c['Exposure'] = i          # opcional pero útil
        df_c['Covar'] = covar
        df_directions_cov_odd_all.append(df_c)

df_directions_odd = pd.concat(df_directions_odd_all, axis=0).reset_index(drop=True)
df_directions_cov_odd = pd.concat(df_directions_cov_odd_all, axis=0).reset_index(drop=True)

#df_directions_odd, df_directions_cov_odd


In [11]:
for covar in ['total_exposomes', 'sociopolitical', 'social_physical', 'GDP', 'number_leng_inst', 'number_stable_inst', 'distance', 'Migration'] + vars_social_physical + vars_sociopolitical:
    c_dfOR = df_directions_odd[df_directions_odd.Covar == covar]
    c_dfOR.to_excel('Results/cross/cross_OR_-with-slovakia_covar-' + covar + '.xlsx')

    print(covar)
    display(c_dfOR)

total_exposomes


,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
0,2.416820,2.766781,2.585887,27.539332,5.939208e-167,Mono,total_exposomes
1,0.741249,0.868727,0.802461,-5.436137,5.444812e-08,One,total_exposomes
2,0.396956,0.466162,0.430169,-20.576111,4.493190e-94,Two,total_exposomes
3,0.534534,0.628138,0.579449,-13.255769,4.178142e-40,Three,total_exposomes
4,0.361242,0.413600,0.386535,-27.528375,8.033882e-167,Total,total_exposomes


sociopolitical


,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
5,2.250224,2.550122,2.395484,27.370740,6.118032e-165,Mono,sociopolitical
6,0.770277,0.898975,0.832142,-4.661940,3.132419e-06,One,sociopolitical
7,0.428368,0.497152,0.461480,-20.356537,4.063090e-92,Two,sociopolitical
8,0.540129,0.627487,0.582172,-14.145702,1.985255e-45,Three,sociopolitical
9,0.391906,0.444183,0.417226,-27.365299,7.101735e-165,Total,sociopolitical


social_physical


...

,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
60,2.078006,2.355758,2.212528,24.813592,6.395452e-136,Mono,inclu_suff_est
61,0.777561,0.907615,0.840075,-4.416855,1.001475e-05,One,inclu_suff_est
62,0.460007,0.535268,0.496212,-18.128216,1.908508e-73,Two,inclu_suff_est
63,0.568696,0.659794,0.612554,-12.930435,3.030997e-38,Three,inclu_suff_est
64,0.424398,0.481164,0.451891,-24.802754,8.372065e-136,Total,inclu_suff_est


cred_elect_est


,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
65,2.284894,2.595828,2.435404,27.347700,1.150097e-164,Mono,cred_elect_est
66,0.769252,0.899176,0.831681,-4.629432,3.666704e-06,One,cred_elect_est
67,0.399046,0.465898,0.431179,-21.289627,1.416505e-100,Two,cred_elect_est
68,0.547987,0.637706,0.591147,-13.590479,4.560860e-42,Three,cred_elect_est
69,0.385069,0.437516,0.410456,-27.336676,1.555290e-164,Total,cred_elect_est


local_dem_est


,2.5%,97.5%,OR,z,P>|z|,Feature,Covar
70,2.128789,2.410649,2.265340,25.778942,1.527533e-146,Mono,local_dem_est
71,0.772047,0.901743,0.834379,-4.570806,4.858514e-06,One,local_dem_est
72,0.443987,0.514708,0.478041,-19.574051,2.573857e-85,Two,local_dem_est
73,0.570601,0.661762,0.614494,-12.878771,5.927181e-38,Three,local_dem_est
74,0.414716,0.469673,0.441340,-25.765073,2.185024e-146,Total,local_dem_est


### 1000-Iteration

In [12]:

results_merge_df_all = data.copy()

covar_list = (
    ['total_exposomes', 'sociopolitical', 'social_physical', 'GDP',
     'number_leng_inst', 'number_stable_inst', 'distance', 'Migration']
    + vars_social_physical
    + vars_sociopolitical
)



n_boosts = 10  #For performance reasons, the number of iterations was set to 10; however, 1,000 iterations were used for the results reported in the manuscript.”
test_size = 500 / results_merge_df_all.shape[0]


df_directions_odd_all = []
df_directions_cov_all = []


results_dict = {}  

for covar in covar_list:

    for boosts in range(n_boosts):

        for i in vars_:

            y_ols = results_merge_df_all["GAP_bin"]
            X_ols = results_merge_df_all[[i, covar]]

            # Split igual al primero: test fijo ~500 y estratificado
            X_train, X_test, y_train, y_test = train_test_split(
                X_ols, y_ols,
                test_size=test_size,
                stratify=y_ols,
                random_state=boosts
            )

            scaler = MinMaxScaler((0.05, 0.95))

            X_train_scaled = pd.DataFrame(
                scaler.fit_transform(X_train),
                columns=X_train.columns,
                index=X_train.index
            )

            X_test_scaled = pd.DataFrame(
                scaler.transform(X_test),
                columns=X_test.columns,
                index=X_test.index
            )


            X_train_scaled["intercept"] = 1
            X_test_scaled["intercept"] = 1


            model = sm.Logit(y_train, X_train_scaled).fit(disp=0)
            y_test_pred = model.predict(X_test_scaled)

            key_y = (covar, i, "y")
            key_p = (covar, i, "ypred")
            if key_y not in results_dict:
                results_dict[key_y] = pd.Series(dtype=float)
                results_dict[key_p] = pd.Series(dtype=float)

            results_dict[key_y] = pd.concat([results_dict[key_y], y_test], axis=0)
            results_dict[key_p] = pd.concat([results_dict[key_p], y_test_pred], axis=0)

            params = model.params
            conf = np.exp(model.conf_int())
            conf["OR"] = np.exp(params)
            conf["z"] = model.tvalues
            conf["P>|z|"] = model.pvalues
            conf.columns = ["2.5%", "97.5%", "OR", "z", "P>|z|"]

            hl10_stat, hl10_p = hosmer_lemeshow(y_test, y_test_pred, g=10)
            hl50_stat, hl50_p = hosmer_lemeshow(y_test, y_test_pred, g=50)
            hl100_stat, hl100_p = hosmer_lemeshow(y_test, y_test_pred, g=100)

            df_i = conf.loc[i:i].copy()
            df_i["Feature"] = i
            df_i["Covar"] = covar
            df_i["boosts"] = boosts

            df_i["hl_statistic_10"] = hl10_stat
            df_i["hl_pvalue_10"] = hl10_p
            df_i["hl_statistic_50"] = hl50_stat
            df_i["hl_pvalue_50"] = hl50_p
            df_i["hl_statistic_100"] = hl100_stat
            df_i["hl_pvalue_100"] = hl100_p

            df_directions_odd_all.append(df_i)

            df_c = conf.loc[covar:covar].copy()
            df_c["Feature"] = i + "(" + covar + ")"
            df_c["Exposure"] = i
            df_c["Covar"] = covar
            df_c["boosts"] = boosts

            df_c["hl_statistic_10"] = hl10_stat
            df_c["hl_pvalue_10"] = hl10_p
            df_c["hl_statistic_50"] = hl50_stat
            df_c["hl_pvalue_50"] = hl50_p
            df_c["hl_statistic_100"] = hl100_stat
            df_c["hl_pvalue_100"] = hl100_p

            df_directions_cov_all.append(df_c)

df_directions_odd = pd.concat(df_directions_odd_all, axis=0).reset_index(drop=True)
df_directions_cov_odd = pd.concat(df_directions_cov_all, axis=0).reset_index(drop=True)

#df_directions_odd, df_directions_cov_odd


In [13]:

features = list(vars_) 
covars = df_directions_odd["Covar"].dropna().unique()

idx = pd.MultiIndex.from_product([covars, features], names=["Covar", "Feature"])
df_directions_odd_f = pd.DataFrame(index=idx)

for c in covars:
    for f in features:
        df_directions_odd_f = summarize_feature_by_covar(
            df_directions_odd, c, f, df_directions_odd_f,
            stat_cols=(10, 50, 100),
            df_offset=2
        )

df_directions_odd_f = df_directions_odd_f.reset_index()
df_directions_odd_f;


In [14]:
for covar in ['total_exposomes', 'sociopolitical', 'social_physical', 'GDP', 'number_leng_inst', 'number_stable_inst', 'distance', 'Migration'] + vars_social_physical + vars_sociopolitical:
    c_dfOR = df_directions_odd_f[df_directions_odd_f.Covar == covar]
    c_dfOR.to_excel('Results/cross/cross_OR_-with-slovakia_covar-' + covar + '-iter.xlsx')

    print(covar)
    display(c_dfOR)

total_exposomes


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR,n_iter
0,total_exposomes,Mono,0.213934,0.265474,0.978753,2.586257,2.579746,2.592768,2.426768,2.760447,10.0
1,total_exposomes,One,0.039509,0.124284,0.958277,0.792143,0.790563,0.793724,0.734806,0.853313,10.0
2,total_exposomes,Two,0.196868,0.209458,0.947697,0.440222,0.439287,0.441157,0.408211,0.474595,10.0
3,total_exposomes,Three,0.096655,0.185198,0.952946,0.576682,0.574951,0.578413,0.533177,0.622410,10.0
4,total_exposomes,Total,0.215037,0.295397,0.980987,0.386452,0.385479,0.387424,0.362041,0.411871,10.0


sociopolitical


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR,n_iter
5,sociopolitical,Mono,0.117155,0.265487,0.976679,2.374267,2.368979,2.379555,2.238941,2.521542,10.0
6,sociopolitical,One,0.064827,0.126897,0.961745,0.815064,0.813525,0.816604,0.757679,0.876410,10.0
7,sociopolitical,Two,0.197457,0.238699,0.983933,0.476165,0.475351,0.476979,0.444670,0.510474,10.0
8,sociopolitical,Three,0.122546,0.199682,0.947545,0.586082,0.584490,0.587674,0.544996,0.629053,10.0
9,sociopolitical,Total,0.124330,0.304223,0.981932,0.420930,0.419993,0.421867,0.396322,0.446391,10.0


social_physical


...

,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR,n_iter
60,inclu_suff_est,Mono,0.268407,0.253758,0.969860,2.189380,2.184384,2.194376,2.064128,2.324932,10.0
61,inclu_suff_est,One,0.089911,0.174509,0.950864,0.821830,0.820318,0.823342,0.764102,0.883828,10.0
62,inclu_suff_est,Two,0.116849,0.213829,0.966461,0.512043,0.511194,0.512893,0.477489,0.549656,10.0
63,inclu_suff_est,Three,0.153625,0.244001,0.949803,0.618753,0.617110,0.620396,0.575830,0.663642,10.0
64,inclu_suff_est,Total,0.288312,0.229510,0.969103,0.456650,0.455608,0.457691,0.430005,0.484376,10.0


cred_elect_est


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR,n_iter
65,cred_elect_est,Mono,0.130412,0.227545,0.973347,2.425724,2.420000,2.431447,2.283487,2.578312,10.0
66,cred_elect_est,One,0.184804,0.218777,0.958116,0.819081,0.817406,0.820756,0.760598,0.881090,10.0
67,cred_elect_est,Two,0.034083,0.260558,0.977119,0.441507,0.440582,0.442431,0.410514,0.475025,10.0
68,cred_elect_est,Three,0.108287,0.247034,0.939445,0.592581,0.590911,0.594250,0.550749,0.636851,10.0
69,cred_elect_est,Total,0.139504,0.240259,0.978546,0.412068,0.411095,0.413041,0.387658,0.437756,10.0


local_dem_est


,Covar,Feature,hl_pvalue_combined_10,hl_pvalue_combined_50,hl_pvalue_combined_100,OR,2.5%,97.5%,minOR,maxOR,n_iter
70,local_dem_est,Mono,0.225678,0.208749,0.975203,2.246335,2.241150,2.251519,2.118677,2.384548,10.0
71,local_dem_est,One,0.090790,0.177728,0.966263,0.813292,0.811863,0.814720,0.755983,0.874766,10.0
72,local_dem_est,Two,0.206672,0.228258,0.981953,0.492654,0.491775,0.493533,0.460213,0.527907,10.0
73,local_dem_est,Three,0.034049,0.154339,0.965985,0.620562,0.618912,0.622212,0.577600,0.665427,10.0
74,local_dem_est,Total,0.227990,0.197388,0.974730,0.445046,0.444019,0.446073,0.419227,0.471882,10.0


In [15]:
dt = time.perf_counter() - t0
m, s = divmod(dt, 60)
print(f"Tiempo total: {int(m)} min {s:.1f} s")

Tiempo total: 4 min 54.6 s
